In [96]:
import pandas as pd
from os import listdir
import tools

PATH = '../data/chromatograms'

In [97]:
data = pd.read_excel('../data/database.xls', skiprows=[0,1,2,3,4,5,6,7,8,9,10])
data.rename(columns = {data.columns[0]: 'status', data.columns[3]: 'number', data.columns[2]: 'date', 'NOM;PRENOM;DDN': 'name', data.columns[10]: 'label'}, inplace=True)
data["status"] = 1
data.head()

,status,name,date,number,AOU (QUANT),UAOROT,UMMA,UAMEVAL,USUCACET,AOU (QUAL),label
0,1,3542_ERNDIM QLOU 2019;D;,01/07/2019 10:01,1900033730,-,-,-,-,-,-,-
1,1,3542_ERNDIM QLOU 2019;E;,01/07/2019 10:06,1900033760,-,-,-,-,-,-,-
2,1,3542_ERNDIM QLOU 2019;F;,01/07/2019 10:10,1900033770,-,-,-,-,-,-,-
3,1,"3542_ERNDIM,DPT 2021;A;",10/03/2021 08:03,2100210312,-,-,-,-,-,-,RAS
4,1,"3542_ERNDIM,DPT 2021;C;",10/03/2021 08:17,2100210394,-,-,-,-,-,30H2MEBut,-


In [98]:
# on élimine les calibrations
data = data[data["name"].str.contains('ERNDIM')==False]
data = data[data["name"].str.contains('Test')==False]

In [99]:
# si pas de tiret ni RAS -> anormal(0)
for i in range(4,10):
    print(data.columns[i])
    data.loc[data[data.columns[i]] != '-', 'status'] = 0
    data.loc[data[data.columns[i]] == 'RAS', 'status'] = 1

# pas d'évidence dans interpretation -> normal
data.loc[data[data.columns[10]].str.contains("Pas d'évidence")==False, 'status'] = 0

AOU (QUANT)
UAOROT
UMMA
UAMEVAL
USUCACET
AOU (QUAL)


In [100]:
normal = data.loc[data['status'] == 1]
normal

,status,name,date,number,AOU (QUANT),UAOROT,UMMA,UAMEVAL,USUCACET,AOU (QUAL),label
43,1,ABDI;ABD ALLAH;08/03/2011,12/10/2020 08:20,2000709604,RAS,-,-,-,-,RAS,Pas d'évidence de trouble d'ordre métabolique ...
44,1,ABDIRAHMAN MOHAMUD;MOHAMED;14/07/2018,08/06/2019 07:00,1900001336,RAS,-,-,-,-,RAS,Pas d'évidence de trouble d'ordre métabolique ...
46,1,ABDULLAH;AHMED NAJI ABDULLAH;15/11/2012,26/04/2021 10:40,2100363385,RAS,-,-,-,-,RAS,Pas d'évidence de trouble d'ordre métabolique ...
52,1,ADOLPHE;RAILEY;12/11/2013,29/11/2019 15:00,1900398354,RAS,-,-,-,-,RAS,Pas d'évidence de trouble d'ordre métabolique ...
53,1,AFONSO;MYA;31/01/2014,09/09/2019 12:00,1900217376,RAS,-,-,-,-,RAS,Pas d'évidence de trouble d'ordre métabolique ...
...,...,...,...,...,...,...,...,...,...,...,...
1126,1,YERNAUX;NICOLAS;16/09/2011,15/12/2020 16:00,2000904612,RAS,-,-,-,-,RAS,Pas d'évidence de trouble d'ordre métabolique ...
1127,1,ZAGHBANI LAMZOURI;FARES;29/06/2016,16/03/2021 08:00,2100230614,RAS,-,-,-,-,RAS,Pas d'évidence de trouble d'ordre métabolique ...
1128,1,ZAID BOUTTEVILLE;THYMEO;27/04/2017,04/12/2020 08:00,2000866233,RAS,-,-,-,-,RAS,Pas d'évidence de trouble d'ordre métabolique ...
1130,1,ZALIMKHANOV;ASKER;13/09/2016,02/02/2021 10:45,2100100830,RAS,-,-,-,-,RAS,Pas d'évidence de trouble d'ordre métabolique ...


In [101]:
not_normal = data.loc[data['status'] == 0]
not_normal

,status,name,date,number,AOU (QUANT),UAOROT,UMMA,UAMEVAL,USUCACET,AOU (QUAL),label
42,0,ABDELHAK;SABRI;29/03/2004,26/02/2021 08:00,2100177136,RAS,-,-,-,-,Présence de métabolites d'origine médicamenteu...,Pas d'évidence de trouble d'ordre métabolique ...
47,0,ABRAHAM;VICTOR;21/12/2012,02/06/2020 17:00,2000357924,RAS,-,-,-,-,Présence de métabolites d'origine bactérienne.,Pas d'évidence de trouble d'ordre métabolique ...
48,0,ABULADZE;QETA;24/02/2018,12/09/2019 08:15,1900224545,RAS,-,-,-,-,Présence de pics inconnus (métabolites d'origi...,Pas d'évidence de trouble d'ordre métabolique ...
49,0,ACHOURI;ASSIA;21/05/2020,09/02/2021 08:00,2100118651,RAS,-,-,-,-,Présence de pics inconnus (métabolites d'origi...,Pas d'évidence de trouble d'ordre métabolique ...
51,0,ADDA;SYLVIO;05/04/1979,13/04/2020 08:00,2000240537,RAS,-,-,-,-,Présence d'un pic inconnu (métabolite d'origin...,Présence d'un pic inconnu (métabolite d'origin...
...,...,...,...,...,...,...,...,...,...,...,...
1133,0,ZEROIL;IMAD;01/06/2012,17/07/2019 15:30,1900079418,-,-,-,-,0.02,-,valeur normale.
1134,0,ZINCIR;SULTAN HAZAL;30/01/2013,11/09/2019 09:50,1900223003,p-OH-phényl-lactate et p-OH-phényl-pyruvate én...,-,-,-,-,Présence importante de métabolites d'origine b...,Suivi d'acidurie homogentisique.
1139,0,ZINGRAFF;ENZO;13/11/2020,04/12/2020 11:00,2000869594,RAS,-,-,-,-,Présence d'un pic inconnu (métabolite d'origin...,Pas d'évidence de trouble d'ordre métabolique ...
1140,0,ZITOUNI;ASSIA;28/12/2009,01/07/2020 15:30,2000434336,Méthylmalonate : 14631 mmol/mol de créat...,-,-,-,-,-,Suivi d'acidurie méthylmalonique.


In [102]:
files = listdir(PATH)
listFiles = []

for i in range(len(files)):
    files[i] = files[i].lower()

# recherche du bon fichier par rapport au numéro qu'on a 
for i in range(len(data)):
    temp = data.iloc[i]
    name = temp['name'].split(';')
    # recherche sur le nom avec la première lettre du prénom
    corresponding = [filename for filename in files if filename.startswith(name[0].lower() + name[1][0].lower()+'-')]
    # vérification que l'on enlève pas trop d'éléments
    #temp2 = [filename for filename in files if filename.startswith(name[0].lower())]
    #if (len(temp2) != len(corresponding)):
        #print('difference :')
        #print(corresponding)
        #[print(i, '  ', name) for i in temp2 if i not in corresponding]
    if(len(corresponding) > 2):
        # si plusieurs possibilités recherche sur la date
        date = temp['date'][:-6].split('/') # récupérer jour mois année uniquement
        date = date[2]+date[1]+date[0] # format du titre sans le jour car il n'est pas toujours bon
        corresponding = [filename for filename in corresponding if date in filename]
        # si on ne trouve pas
        if(len(corresponding) > 2 or len(corresponding) < 2):
            print("Problème date : ", name[0])
            listFiles.append('')
            continue
    elif len(corresponding) < 2:
        #print("Problème nom : ", name[0])
        listFiles.append('')
        continue
    # parmis les deux éléments on prend le chromatogramme
    if tools.CHROM_EXT.lower() in corresponding[0]:
        file = corresponding[0]
    else :
        file = corresponding[1]
    # on ajoute le nom du fichier à la base si il n'y est pas déjà
    filename = file[:-len(tools.CHROM_EXT)]
    if filename in listFiles:
        print('Problème de bouble sélection : ', name[0])
        listFiles.append('')
    else:
        listFiles.append(filename)
    

data['file'] = listFiles

data = data.loc[data['file'] != '']
print('Data : ', len(data))

Problème nom :  ABDELHAK
Problème nom :  ABDI
Problème nom :  ABDIRAHMAN MOHAMUD
Problème nom :  ABDULLAH
Problème nom :  ABULADZE
Problème nom :  ACHOURI
Problème nom :  ADOLPHE
Problème nom :  AFONSO
Problème nom :  AGDGOMELASHVILI
Problème nom :  AGKHANOV
Problème nom :  AHAMADA
Problème nom :  AIT BADDOU
Problème nom :  AKESBI
Problème nom :  AKIN
Problème nom :  ALIAJ
Problème nom :  ALIJAJ
Problème nom :  ALOTTO DUCHENE
Problème nom :  AMIRI
Problème nom :  AMIRI
Problème nom :  ANCEL
Problème nom :  ANCLIN
Problème nom :  ANSOULD
Problème nom :  ANTOINE
Problème nom :  ANTONI WATELET
Problème nom :  ANTZEMBERGER
Problème nom :  AOURDOU NAIT HADDOU
Problème nom :  ARNOLD
Problème nom :  ARSLAN
Problème nom :  ASANLLARI
Problème nom :  ASATRYAN
Problème nom :  AUBERTIN
Problème nom :  AURARD
Problème nom :  AVDALYAN
Problème nom :  AZIRI
Problème nom :  BA
Problème nom :  BAECHEL PICHECA
Problème nom :  BAI
Problème nom :  BALOZIAN
Problème nom :  BARAKAT KASSIM
Problème nom :  BA

In [103]:
save = data[["file", "status", "label"]]
save.to_csv("../data/database.csv", index=False)